In [ ]:
import cv2
import numpy as np
import math
from numpy.linalg import inv

def transformPoints(t):   #If it is inlier then return 1, else return 0.This func for counting
    xPrime=float(t[0]/t[2])
    yPrime=float(t[1]/t[2])
    
    for index in range(len(dst_points)):
        if (math.sqrt((xPrime - dst_points[index][0])**2 + (yPrime - dst_points[index][1])**2 ) <= 3.0):
            return 1                
    return 0      


def random4points(): #Find randomly 4 points from the matches keypoints
    random=np.random.randint(0,len(src_points))
    random2=np.random.randint(0,len(src_points))
    random3=np.random.randint(0,len(src_points))
    random4=np.random.randint(0,len(src_points))
    
    src_random=[]
    dst_random=[]
    
    src_random.append((src_points[random]))
    src_random.append((src_points[random2]))
    src_random.append((src_points[random3]))
    src_random.append((src_points[random4]))
    
    dst_random.append((dst_points[random]))
    dst_random.append((dst_points[random2]))
    dst_random.append((dst_points[random3]))
    dst_random.append((dst_points[random4]))

    return np.array(src_random),np.array(dst_random)

def updateRansac(inlier): #TO update Ransac N value
    
    w=float(inlier/ len (src_points))
    ransac= -2.0 / float(math.log10 ( 1.0 - (w)**4 ))
    return int(ransac - 1)

def dlt(src_points,dst_points):  #Direct Linear Translation function returns homography
    average_src_x =0
    average_src_y =0
    average_dst_x =0
    average_dst_y =0
   
    for i in range(len(src_points)):
        average_src_x+=src_points[i][0]
        average_src_y+=src_points[i][1]
        average_dst_x+=dst_points[i][0]
        average_dst_y+=dst_points[i][1]
    
    average_src_x= float(average_src_x / len(src_points))
    average_src_y= float(average_src_y / len(src_points))
    average_dst_x= float(average_dst_x / len(src_points))
    average_dst_y= float(average_dst_y / len(src_points))

    new_src_points=[]
    new_dst_points=[]
 
    for i in range(len(src_points)):

        x1=src_points[i][0]-average_src_x
        y1=src_points[i][1]-average_src_y
        x2=dst_points[i][0]-average_dst_x
        y2=dst_points[i][1]-average_dst_y
    
        new_src_points.append([x1,y1])
        new_dst_points.append([x2,y2])
    
    distancesrc=0
    distancedst=0
    
    for i in range(len(new_src_points)):
        distancesrc += float(math.sqrt(new_src_points[i][0]**2 +   new_src_points[i][1]**2))
        distancedst += float(math.sqrt(new_dst_points[i][0]**2 +   new_dst_points[i][1]**2))
  
    lamda=  float( math.sqrt(2)/(distancesrc))
    lamda2=  float( math.sqrt(2)/(distancedst))
   
    t=np.array([[float(lamda),0,float(lamda)*(-average_src_x)],[0,float(lamda),float(lamda)*(-average_src_y)],[0,0,1]])
    t2=np.array([[float(lamda2),0,float(lamda2)*(-average_dst_x)],[0,float(lamda2),float(lamda2)*(-average_dst_y)],[0,0,1]])
    listA=[]
    
    for i in range(len(new_src_points)):
        new_src_points[i][0]= lamda * new_src_points[i][0]
        new_src_points[i][1]= lamda * new_src_points[i][1]
        new_dst_points[i][0]= lamda2 * new_dst_points[i][0]
        new_dst_points[i][1]= lamda2 * new_dst_points[i][1]      
    
    for i in range(len(src_points)):
        temp = np.dot (t,np.array([[src_points[i][0]],[src_points[i][1]],[1.0]]))
        
        temp[0] = temp[0] /temp[2]
        temp[1] = temp[1] /temp[2]
        temp=np.array(temp)
     
        temp2 =  np.dot (t2,np.array([[dst_points[i][0]],[dst_points[i][1]],[1.0]]))
        temp2[0] = temp2[0] /temp2[2]
        temp2[1] = temp2[1] /temp2[2]
        
        temp2=np.array(temp2)
        new_src_points[i][0]= temp[0]
        new_src_points[i][1]= temp[1]
        
        
        new_dst_points[i][0]= temp2[0]
        new_dst_points[i][1]= temp2[1]
    
    new_src_points=np.array(new_src_points)
    new_dst_points=np.array(new_dst_points)
    
    for i in range(len(new_src_points)):
        listA.append([0,0,0,-new_src_points[i][0],-new_src_points[i][1],-1.0,new_dst_points[i][1]*new_src_points[i][0],new_dst_points[i][1]*new_src_points[i][1],new_dst_points[i][1]])
        listA.append([new_src_points[i][0],new_src_points[i][1],1.0,0.0,0.0,0.0,-new_dst_points[i][0]*new_src_points[i][0],-new_src_points[i][1]*new_dst_points[i][0],-new_dst_points[i][0]])
    listA=np.array(listA)

    w,u,vt=cv2.SVDecomp(np.asarray(listA))
    homography=[[vt[-1][0],vt[-1][1],vt[-1][2]],[vt[-1][3],vt[-1][4],vt[-1][5]],[vt[-1][6],vt[-1][7],vt[-1][8]]]
    invT = inv(t2)
    newHomograpy=np.dot(invT,np.array(homography))
    final=np.dot(np.array(newHomograpy),t)
    return np.array(final)


def findInlier(lastHomography,src_points,dst_points): #Find nearest inlier to the point, append to new list and return
    listFinalsrc=[]
    listFinaldst=[]
    indeks=0
    x=0
    for p1 in src_points:
        y=0
        flag=False
        lowest=3.0
        p=np.dot(lastHomography,np.array([[p1[0]],[p1[1]],[1.0]]))
        xp=p[0]/p[2]
        yp=p[1]/p[2]
        
        for p2 in dst_points:
            t=float(math.sqrt((xp[0] - p2[0])**2 + (yp[0] - p2[1])**2 ))
            if (t <= 3.0):
                if (lowest>t):
                    lowest=t
                    indeks=y
                    flag=True  
            y+=1
        if(flag==True):
            listFinalsrc.append(p1)
            listFinaldst.append(dst_points[indeks])
        x+=1
    return  np.array(listFinalsrc),np.array(listFinaldst)



In [ ]:
src_points=[]
dst_points=[]
img1=cv2.imread("img1.jpg")
img2 = cv2.imread('img2.jpg')

# trainImage
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create(contrastThreshold = 0.04 ,edgeThreshold = 10)
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)


temp=0   
for i in range(len(des1)):               #Match them by using nearest neighbour
    distance=100000000

    temp=0
    for y in range(len(des2)):
        x=0
        for a in range(len(des1[0])):
            x+=math.sqrt((des1[i][a]-des2[y][a])**2) 
          
        if(x<distance):
            distance=x
            temp=int(y)
    src_points.append(kp1[i].pt)    #Our correspondences in src and dst points, It takes time to calculate
    dst_points.append(kp2[temp].pt)
    
print ("KEYPOINTS",len(src_points))
Nransac=10000  
i=0
beforeinlier=0

while(i<Nransac):    #Starting of RANSAC
    inlier=0
    inlierRatio=0
    src,dst=random4points()   # 4 random point
    homography=dlt(src,dst)   #Estimating Homography
   
    for points in (src_points):     #IN every src_point multiplied by homography and checks whether in dst list is there a pixel distance < 3.
    
        t=np.dot(homography,np.array([[points[0]],[points[1]],[1.0]]))
        inlier+=transformPoints(t)
      
    inlierRatio= float(inlier / len(src_points)) #Inlier Ratio
    if (inlier>=beforeinlier):
        Nransac = updateRansac(inlier) #By giving number of inlier, we update N value
        beforeinlier=inlier #This is for the copy operation to check inlier's
        i=0   #Just reset the loop whenever N updates
        lastHomography=np.copy(homography) #Save the last and best homography
        print ("Number of inlier is",inlier)
        print ("Ransac",Nransac)
        print ("Inlier ratio is ",inlierRatio * 100)
        
    
    print ("Loading  ",(i/Nransac)*100)
    i+=1

print ("LAST OF LAST homography",lastHomography) #Best homography yet

l1,l2=findInlier(lastHomography,src_points,dst_points) #Find inlier with the best homography.


copy=0
i=0 #Just find better homography 5 times by inliers
while(i<5):   #This should be infinite loop until it converges but our list will be full of same points. I could not understand why ??
    print ("l1",len(l1))
    print ("l2",len(l2))
    copy=len(l1)
    lH=dlt(l1,l2)#Update to estimate better homography
 
    print ("Before INLIER ",len(l1))
    l1,l2=findInlier(lH,l1,l2) #To get better and realistic inlier until it converges.
    print ("After INLIER ",len(l1))
    print (copy,len(l1))
    if copy==(len(l1)):
        break
   
    i+=1

    

topLeft=np.dot(lH,np.array([[160.0],[160.0],[1.0]]))

topLeft[0]=float(topLeft[0]/topLeft[2])
topLeft[1]=float(topLeft[1]/topLeft[2])


botRight=np.dot(lH,np.array([[455.0],[576.0],[1.0]]))

botRight[0]=float(botRight[0]/botRight[2])
botRight[1]=float(botRight[1]/botRight[2])

cv2.rectangle(img2,(topLeft[0],topLeft[1]),(botRight[0],botRight[1]),(0,0,255),2)

cv2.imwrite("object_detection.png",img2)


In [ ]:
#### COMMENT 1 ####



#ORB 
#ORB uses by the FAST key point detector and BRIEF  descriptor
#Then a Harris corner measure is  applied  to  find  top  N  points.    FAST  does  not  compute  the 
#orientation  and  is  rotation  variant.  It  computes  the  intensity weighted  centroid  
#of  the  patch  with  located  corner  at  center. 

#SIFT algorithm uses Difference of Gaussians

#SIFT  proposed  by  Lowe  solves  the  image  rotation,  affine  transformations,  intensity,  and  viewpoint  change  in  matching 
#features.A key point localization where the key  point  candidates  are  localized  and  refined  by  eliminating 

#Descriptor to compute the local image descriptor for each   key   point   based   on   image   gradient   magnitude   and orientation 

#I understood that;
 
#BRIEF performs poorly with rotation.

#ORB is the fastest but SIFT is more succes rate. --> Comparing the images with varying intensity or
#comparing the image with its rotated image

#With  scaled image ORB is faster and more succesfull than SIFT


# As I understood briefly from the source:
# https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_orb/py_orb.html
# https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_sift_intro/py_sift_intro.html?highlight=sift

In [ ]:
#### COMMENT 2 ####
# In SIFT we use Euclidean distance. Every keypoint has length of 128 size
#there will be Keypoint x 128. We calculate as follow
# (x0-x'0)**2 + (x1-x'1) +....+ x128-x'128) and square root of them

#In ORB we use Hamming distance. We use XOR operation to catch the difference
#So, between 2 keypoint distance we count (1 values )the differences between keypoints that we gather by XOR operation




In [ ]:
#### COMMENT 3 ####
# Our points of set will be decreased because our distance will lower than 3.0 pixel distance. We can think it as a circle
# radius and our point number will decreased since the radius is decreased but the results could be more realistic.
# if we decrease too much than our set of points  decreased too much and no longer to find good matching points

#If we decrease our number of inlier will decrease but we use nearest neighbour so somepoints will remain the same

In [ ]:
#### COMMENT 4 ####
# Normalization is basically putting both points on the border which makes the solution more stable
#It's a matter of numerical accuracy. By normalizing the data set, we center data and give it unit variance.
#We move the image center to the (0,0)
#Normalization is essential not only for numerical stability, but also for more accurate estimation in presence of noise and faster solution
#Normalization affects  SVD computation.It is better and to obtain more accurate estimate.